In [1]:
inputPath1 = "/data/students/bigdata_internet/lab3/register.csv"

In [2]:
inputPath2 = "/data/students/bigdata_internet/lab3/stations.csv"

In [3]:
rdd = sc.textFile(inputPath1)

In [4]:
header = rdd.first()

In [5]:
new_rdd = rdd.filter(lambda x: x!= header)

In [6]:
Register = new_rdd.map(lambda line : line.split("\t"))

In [7]:
#1.1.1 number of all rdds without any filter

In [8]:
Register.count()

25319028

In [9]:
condition = lambda x:not (x[2] == '0' and x[3]== '0')

In [10]:
filteredRdd = Register.filter(condition)

In [11]:
#1.1.1 number of rdds after removing wrong data due to temporary problems of the monitoring system

In [12]:
filteredRdd.count()

25104121

In [13]:
# change to timeslot(day of week and hour) Tj

In [14]:
from datetime import datetime

def changeToDay(timestamp, line):
    f = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    weekDay = f.strftime("%A")  # %A gives the full weekday name
    hour = f.hour
    return [line[0], (weekDay, hour), line[2], line[3]]

In [15]:
formattedRdd = filteredRdd.map(lambda x: changeToDay(x[1],x))

In [16]:
stations = formattedRdd.map(lambda x : ((x[0],x[1]),1))

In [17]:
aggStations= stations.reduceByKey(lambda v1, v2: v1+v2)

In [18]:
criticalStations = formattedRdd.filter(lambda x: x[3] == '0')

In [19]:
a = criticalStations.map(lambda x : ((x[0],x[1]),1))

In [20]:
SiTj = a.reduceByKey(lambda v1, v2: v1+v2)

In [21]:
joinPairRdd = aggStations.join(SiTj)

In [22]:
Csitj = joinPairRdd.map(lambda x: ((x[0], x[1]), x[1][1] / x[1][0]))

In [23]:
Csitj.take(5) #2.1 for each pair the criticality value is calculated

[((('108', ('Monday', 18)), (586, 10)), 0.017064846416382253),
 ((('108', ('Monday', 22)), (596, 37)), 0.06208053691275168),
 ((('108', ('Tuesday', 1)), (354, 66)), 0.1864406779661017),
 ((('108', ('Tuesday', 3)), (360, 30)), 0.08333333333333333),
 ((('108', ('Tuesday', 5)), (570, 59)), 0.10350877192982456)]

In [24]:
sortedCsitj = Csitj.sortBy(lambda x: (x[1], int(x[0][0][0]), x[0][0][1][0], x[0][0][1][1]))
#2.3 now orderin them by criticality value,station id,day of week and hour

In [25]:
aboveThreshold = sortedCsitj.filter(lambda x:x[1]>0.6)

In [26]:
registerations = aboveThreshold.map(lambda x : [x[0][0][0],(x[0][0][1][0],x[0][0][1][1],x[1])])
#2.2 only critical pairs with above minimum criticality threshold(0.6) are selected

In [27]:
registerations.collect()

[['9', ('Friday', 10, 0.6129032258064516)],
 ['10', ('Saturday', 0, 0.622107969151671)],
 ['58', ('Monday', 1, 0.6239554317548747)],
 ['9', ('Friday', 22, 0.6258389261744967)],
 ['58', ('Monday', 0, 0.6323119777158774)]]

In [28]:
rdd_location = sc.textFile(inputPath2)
header2 = rdd_location.first()
new_rdd2 = rdd_location.filter(lambda x: x!= header2)
Register2 = new_rdd2.map(lambda line : line.split("\t"))

In [29]:
Register2.take(5)

[['1', '2.180019', '41.397978', 'Gran Via Corts Catalanes'],
 ['2', '2.176414', '41.394381', 'Plaza TetuÃ¡n'],
 ['3', '2.181164', '41.393750', 'Ali Bei'],
 ['4', '2.181400', '41.393364', 'Ribes'],
 ['5', '2.180214', '41.391072', 'Pg LluÃ\xads Companys']]

In [30]:
PairRDDregister = Register2.map(lambda x: (x[0],(x[1],x[2],x[3])))

In [31]:
joined=registerations.join(PairRDDregister)

In [32]:
final = joined.sortBy(lambda x: x[1][0][2])

In [ ]:
final

In [33]:
#2.4 storing them with header and columns are separated by tab

In [34]:
formatted_rdd = final.map(lambda x: "\t".join([
    x[0],                           # station
    x[1][0][0],                      # day of week
    str(x[1][0][1]),                 # hour
    str(x[1][0][2]),                 # criticality value
    x[1][1][0],                      # station longitude
    x[1][1][1],                      # station latitude
    x[1][1][2]                       # station name
]))

In [40]:
formatted_rdd.collect()

["9\tFriday\t10\t0.6129032258064516\t2.185294\t41.385006\tMarquÃ¨s de l\\'Argentera",
 '10\tSaturday\t0\t0.622107969151671\t2.185206\t41.384875\tAvinguda del Marques Argentera',
 '58\tMonday\t1\t0.6239554317548747\t2.170736\t41.377536\tSant Oleguer',
 "9\tFriday\t22\t0.6258389261744967\t2.185294\t41.385006\tMarquÃ¨s de l\\'Argentera",
 '58\tMonday\t0\t0.6323119777158774\t2.170736\t41.377536\tSant Oleguer']

In [35]:
outputFile = "/user/s310279/lab3/outPutByRDD"

In [36]:
header2write = "station\tstation longitude\tstation latitude\tday of week\thour\tcriticality value"

In [37]:
final_rdd = formatted_rdd.mapPartitions(lambda iterator: [header2write] + list(iterator))

In [41]:
final_rdd.saveAsTextFile(outputFile)